In [ ]:
import numpy as np
import pandas as pd
import sys
import time
import tabula
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# chrome_options = Options()
# chrome_options.add_argument("--headless")

In [ ]:
# import requests
# url = "https://www.crowley.com.br/cbab/relatorios/mailing_anual/TOP100BrasilTheCrowleyOfficialBroadcast2021.pdf"
# response = requests.get(url)
# response = requests.get(url.get('href'))
# pdf = open("pdf1.pdf", 'wb')
# pdf.write(response.content)
# pdf.close()

In [ ]:
# import requests
# import os
# url = "https://www.crowley.com.br/cbab/relatorios/mailing_anual/TOP100BrasilTheCrowleyOfficialBroadcast2021.pdf"
# response = requests.get(url, stream=True)

# # isolate PDF filename from URL
# pdf_file_name = os.path.basename(url)
# if response.status_code == 200:
#     # Save in current working directory
#     filepath = os.path.join(os.getcwd(), pdf_file_name)
#     with open(filepath, 'wb') as pdf_object:
#         pdf_object.write(response.content)
#         print(f'{pdf_file_name} was successfully saved!')
# else:
#     print(f'Uh oh! Could not download {pdf_file_name},')
#     print(f'HTTP response status code: {response.status_code}')

In [ ]:
file_path = "/Users/jonas/Documents/GitHub/get_musics/TOP100BrasilTheCrowleyOfficialBroadcast2021.pdf"
arquivo_pdf = tabula.read_pdf(file_path, pages='all')
df = arquivo_pdf[0]
df.rename(columns={df.columns[0]: "Ranking", df.columns[1]: "Titulo", df.columns[2]: "Artista", df.columns[3]: "Selo", df.columns[4]: "Execucoes"}, inplace=True)
a = arquivo_pdf[2]
a.rename(columns={a.columns[0]: "Ranking", a.columns[1]: "Titulo", a.columns[2]: "Artista", a.columns[3]: "Selo", a.columns[4]: "Execucoes"}, inplace=True)
a.Ranking = pd.to_numeric(a.Ranking)
df = df.loc[df.index[3:]]
df.Ranking = pd.to_numeric(df.Ranking)
df = df.append(a)
df.sort_values(by=['Ranking'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(10)

In [ ]:
df['Pesquisa'] = df['Titulo'] + " " + df['Artista']
df['Pesquisa'] = df['Pesquisa'].str.replace('LIVE', '')
# df.head()

In [ ]:
df['Letra'] = ''
# df.head()

In [ ]:
pesquisa = df['Pesquisa'].tolist()

In [ ]:
capa = DesiredCapabilities.CHROME

capa["pageLoadStrategy"] = "none"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), desired_capabilities=capa)

actions = ActionChains(driver)

wait = WebDriverWait(driver, 20)

url = "https://www.google.com".encode('ascii', 'ignore').decode('unicode_escape')


In [ ]:
for music in pesquisa:
    musica = str(music)
    musica = musica.encode('ascii', 'ignore').decode('unicode_escape')
    driver.get(url)
    wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')))
    driver.execute_script("window.stop();")
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(musica)
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    time.sleep(3)
    elems = driver.find_elements(By.XPATH, "//a[@href]")
    for elem in elems:
        if "https://www.letras.mus.br" in elem.get_attribute("href"):
            web = str(elem.get_attribute("href")).encode('ascii', 'ignore').decode('unicode_escape')
            driver.get(url=web)
            try:
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "cnt-letra")))
                driver.execute_script("window.stop();")
                letra = driver.find_element(By.CLASS_NAME, "cnt-letra").text
                df.loc[df[df['Pesquisa'] == musica].index[0], "Letra"] = letra
            except:
                print(Exception)
                print(music)
            break

In [ ]:
# Pesquisar no google titulo + artista
# Usar o soup para pegar os links
# Pegar o link que começa com letras.mus.br
# Entrar no link e pegar a letra da musica